In [98]:
import pandas as pd
import re
from bs4 import BeautifulSoup

In [79]:
# Load the dataset
df = pd.read_pickle('dataset_raw.pkl')

df.head()

,extracted_data
id,
386123400,"<html><head><meta http-equiv=\""Content-Type\"" ..."
386123500,"<html><head><meta http-equiv=\""Content-Type\"" ..."
386129400,"<html><head><meta http-equiv=\""Content-Type\"" ..."
386132200,"<html><head><meta http-equiv=\""Content-Type\"" ..."
386133100,"<html><head><meta http-equiv=\""Content-Type\"" ..."


In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 446094 entries, 386123400 to 895772200
Data columns (total 1 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   extracted_data  446094 non-null  object
dtypes: object(1)
memory usage: 6.8+ MB


In [81]:
BAM_regex = r'B[ ]?[ÖO][ ]?L[ ]?G[ ]?E A[ ]?D[ ]?L[ ]?[İI][ ]?Y[ ]?E M[ ]?A[ ]?H[ ]?K[ ]?E[ ]?M[ ]?E[ ]?S[ ]?[Iİ]'

count = df[df['extracted_data'].str.contains(BAM_regex)].shape[0]
print(count)

128416


In [82]:
# Discarding "Bölge Adliye Mahkemesi" records, since they they differ in formating
df = df[~df['extracted_data'].str.contains(BAM_regex)]

In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 317678 entries, 386123400 to 895772200
Data columns (total 1 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   extracted_data  317678 non-null  object
dtypes: object(1)
memory usage: 4.8+ MB


In [84]:
query_regex = r'<br>([^<]*dava dilekçe[^<]*)<br>(.*)'

count = df[df['extracted_data'].str.contains(query_regex)].shape[0]
print(count)

C:\Users\Stamina\AppData\Local\Temp\ipykernel_21944\1737465957.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  count = df[df['extracted_data'].str.contains(query_regex)].shape[0]


275817


In [85]:
# Discarding rows that dont have our desired query format
df = df[df['extracted_data'].str.contains(query_regex)]

C:\Users\Stamina\AppData\Local\Temp\ipykernel_21944\3536442561.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[df['extracted_data'].str.contains(query_regex)]


In [91]:
# Extract the query and document pairs
query_doc_df = df['extracted_data'].str.extract(query_regex)
query_doc_df.columns = ['query', 'document']

query_doc_df.head()

,query,document
id,,
386123400,DAVA: Davacı vekili dava dilekçesinde özetle; ...,DEĞERLENDİRME:Dava dilekçesi ve sair tüm evrak...
386123500,Davacı vekilinin mahkememize verdiği dava dile...,Delillerin Değerlendirilmesi :<br>Tüm dosya ka...
386129400,Davacı şirket yetkilisi dava dilekçesinde özet...,Dosya kapsamında yer alan kayıt ve belgeler in...
386132200,Davacı vekili dava dilekçesinde özetle; Keşide...,DELİLLER VE GEREKÇE/<br>Davacı vekilinin 19/01...
386172900,Davacı vekili dava dilekçesinde özetle; müvekk...,DELİLLER VE GEREKÇE / <br>... ATM'nin ... Esa...


In [92]:
query_doc_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 275817 entries, 386123400 to 895772200
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   query     275817 non-null  object
 1   document  275817 non-null  object
dtypes: object(2)
memory usage: 6.3+ MB


In [112]:
# Remove html tags from the data

query_doc_df = query_doc_df.map(lambda s: BeautifulSoup(s).text)
# query_doc_df = query_doc_df.map(lambda x: re.sub('<.*?>', '', str(x))) # an alternative, might be less robust

C:\Users\Stamina\AppData\Local\Temp\ipykernel_21944\2830717337.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  query_doc_df = query_doc_df.map(lambda s: BeautifulSoup(s).text)


In [113]:
# At this point, we've eliminated BAM judgments, extracted query-document pairs and removed the HTML tags. 

query_doc_df.head()

,query,document
id,,
386123400,DAVA: Davacı vekili dava dilekçesinde özetle; ...,DEĞERLENDİRME:Dava dilekçesi ve sair tüm evrak...
386123500,Davacı vekilinin mahkememize verdiği dava dile...,Delillerin Değerlendirilmesi :Tüm dosya kapsam...
386129400,Davacı şirket yetkilisi dava dilekçesinde özet...,Dosya kapsamında yer alan kayıt ve belgeler in...
386132200,Davacı vekili dava dilekçesinde özetle; Keşide...,DELİLLER VE GEREKÇE/Davacı vekilinin 19/01/201...
386172900,Davacı vekili dava dilekçesinde özetle; müvekk...,DELİLLER VE GEREKÇE / ... ATM'nin ... Esas sa...


In [114]:
# Let's save our dataset in this state into a pickle
query_doc_df.to_pickle('dataset_qdpairs.pkl')